In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("data.csv")

In [3]:
df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26.0,1,1-2 Year,No,30170.0,26.0,88,0
381105,381106,Male,30,1,37.0,1,< 1 Year,No,40016.0,152.0,131,0
381106,381107,Male,21,1,30.0,1,< 1 Year,No,35118.0,160.0,161,0
381107,381108,Female,68,1,14.0,0,> 2 Years,Yes,44617.0,124.0,74,0


In [5]:
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [18]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [61]:
from sklearn.model_selection import train_test_split
X = df.drop('Response',axis=1)

y = df['Response']
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.2,random_state=42)

In [37]:
num_columns = [col for col in X.columns if df[col].dtype != 'O']
cat_columns = [col for col in X.columns if df[col].dtype == 'O']

In [38]:
pipelines = ColumnTransformer([
    ('num_pipeline',StandardScaler(),num_columns),
    ('cat_pipeline',OneHotEncoder(),cat_columns)
])

In [63]:
preprocessed_X_train=pipelines.fit_transform(train_X)
preprocessed_X_test=pipelines.transform(test_X)
preprocessed_X_train.dtype

dtype('float64')

In [65]:
preprocessed_X_test[0]

array([ 0.087518  , -1.14997813,  0.04664784, -1.77132284,  1.08702755,
       -0.58906482,  0.88573533, -0.98362499,  1.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ])

In [66]:
pipelines.get_feature_names_out()


array(['num_pipeline__id', 'num_pipeline__Age',
       'num_pipeline__Driving_License', 'num_pipeline__Region_Code',
       'num_pipeline__Previously_Insured', 'num_pipeline__Annual_Premium',
       'num_pipeline__Policy_Sales_Channel', 'num_pipeline__Vintage',
       'cat_pipeline__Gender_Female', 'cat_pipeline__Gender_Male',
       'cat_pipeline__Vehicle_Age_1-2 Year',
       'cat_pipeline__Vehicle_Age_< 1 Year',
       'cat_pipeline__Vehicle_Age_> 2 Years',
       'cat_pipeline__Vehicle_Damage_No',
       'cat_pipeline__Vehicle_Damage_Yes'], dtype=object)

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': [2,3,4,5,6,7,10],
               'min_samples_leaf': [4, 6, 8],
               'min_samples_split': [5, 7,10],
               'n_estimators': [300]}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
model.fit(preprocessed_X_train,train_y)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


,estimator,RandomForestClassifier()
,param_distributions,"{'criterion': ['entropy', 'gini'], 'max_depth': [2, 3, ...], 'min_samples_leaf': [4, 6, ...], 'min_samples_split': [5, 7, ...], ...}"
,n_iter,10
,scoring,None
,n_jobs,-1
,refit,True
,cv,4
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,101
,error_score,nan


In [69]:
best_params = model.best_params_

print("Best Hyperparameters:")
print(best_params)

Best Hyperparameters:
{'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_depth': 3, 'criterion': 'gini'}


In [70]:
import pickle
#with automatically closes the file afer the block is executed
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)